In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [3]:
import tensorflow as tf
# this get the images we want
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# data preprocessing part 1
#data augmentation
# every pixel in the image is 0-1 this only applied to training data set
train_datagen = ImageDataGenerator(rescale= 1./255,
                                  shear_range= 0.2,
                                  zoom_range=0.2,
                                  horizontal_flip = True)

In [6]:
training_set = train_datagen.flow_from_directory('/kaggle/input/dogs-cats-images/dataset/training_set',
                                                target_size = (64,64),
                                                batch_size= 32,
                                                class_mode = 'binary')
# as this is a binary classification binary is given othervise 'category' should be given

Found 8000 images belonging to 2 classes.


In [7]:
test_datagen  = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/kaggle/input/dogs-cats-images/dataset/test_set',
                                                target_size = (64,64),
                                                batch_size= 32,
                                                class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [8]:
from tensorflow.keras.layers import Conv2D

In [13]:
#part 2 building the CNN

# initialising the CNN
cnn = tf.keras.models.Sequential()

#convolution step1
# in padding same for the 0 padding and void for other according to the model it will taken 
# input shape is 64*64 the size of the image and channels 3 as the rgb
cnn.add(tf.keras.layers.Conv2D(filters=32, padding= 'same', kernel_size = 3, activation = 'relu', input_shape = [64,64,3]))

#poooling step2
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides =2))

#another convo layer
cnn.add(tf.keras.layers.Conv2D(filters= 32, padding='same', kernel_size=3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size =2, strides = 2))

#flattening - step 3 making ready to ann
cnn.add(tf.keras.layers.Flatten())

#full connection step4 ann
cnn.add(tf.keras.layers.Dense(units=128, activation = 'relu'))

# output layer step 5
cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))


In [14]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [15]:
# Training the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'])

#fit the model
cnn.fit(x=training_set, validation_data = test_set, epochs=10)

Epoch 1/10
250/250 [==============================] - 114s 454ms/step - loss: 0.6865 - accuracy: 0.5516 - val_loss: 0.5826 - val_accuracy: 0.7060
Epoch 2/10
250/250 [==============================] - 59s 237ms/step - loss: 0.5892 - accuracy: 0.6862 - val_loss: 0.5415 - val_accuracy: 0.7320
Epoch 3/10
250/250 [==============================] - 60s 238ms/step - loss: 0.5439 - accuracy: 0.7230 - val_loss: 0.5492 - val_accuracy: 0.7080
Epoch 4/10
250/250 [==============================] - 59s 238ms/step - loss: 0.5296 - accuracy: 0.7345 - val_loss: 0.4765 - val_accuracy: 0.7840
Epoch 5/10
250/250 [==============================] - 60s 240ms/step - loss: 0.4922 - accuracy: 0.7621 - val_loss: 0.5784 - val_accuracy: 0.7325
Epoch 6/10
250/250 [==============================] - 59s 237ms/step - loss: 0.4569 - accuracy: 0.7838 - val_loss: 0.4852 - val_accuracy: 0.7715
Epoch 7/10
250/250 [==============================] - 59s 234ms/step - loss: 0.4491 - accuracy: 0.7905 - val_loss: 0.4721 - val_a

In [16]:
#save the model in h5 format which is suitable for keras
#as the model

from tensorflow.keras.models import load_model
cnn.save('model_rcat_dog.h5')

In [17]:
#testing fgr new data for a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/kaggle/input/dogs-cats-images/dataset/test_set/dogs/dog.4781.jpg', target_size = (64,64))
#convert the image to array
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)

In [18]:
result

array([[0.9851175]], dtype=float32)

In [19]:
if result[0] <= 0.5:
    print('the image is a cat')
else:
    print('the image is a dog')

the image is a dog
